# 14 - Diagnóstico da Anomalia de Previsão Nula

Este notebook investiga por que as previsões zeram nas últimas semanas de janeiro,
analisando especificamente como as features de lag se comportam durante a previsão.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print('🔍 Iniciando Diagnóstico da Anomalia de Previsão Nula')
print('🎯 Objetivo: Identificar por que previsões zeram nas últimas semanas')

## 1. Carregar Dados de Teste do Pipeline Final

In [ ]:
# Tentar carregar dados do notebook 13 se disponível
try:
    dados_teste = pd.read_parquet('../data/submissao3/dados_teste_com_previsoes.parquet')
    print(f'📂 Dados de teste carregados: {dados_teste.shape}')
    print(f'📊 Período: {dados_teste["semana"].min()} até {dados_teste["semana"].max()}')
except FileNotFoundError:
    print('⚠️ Arquivo de dados de teste não encontrado.')
    print('💡 Execute primeiro o notebook 13 para gerar os dados de teste com previsões.')
    print('🔄 Criando dados simulados para demonstração...')
    
    # Criar dados simulados para demonstração
    semanas_janeiro = pd.date_range('2023-01-02', periods=5, freq='W-MON')
    dados_teste = pd.DataFrame({
        'semana': np.repeat(semanas_janeiro, 1000),
        'pdv_id': np.tile(range(1, 201), 25),
        'produto_id': np.tile(range(1001, 1006), 1000),
        'quantidade_prevista': np.random.uniform(0, 10, 5000),
        'probabilidade_venda': np.random.uniform(0.1, 0.9, 5000)
    })
    
    # Simular anomalia nas últimas 2 semanas
    mask_ultimas_semanas = dados_teste['semana'].isin(semanas_janeiro[-2:])
    dados_teste.loc[mask_ultimas_semanas, 'quantidade_prevista'] = 0
    dados_teste.loc[mask_ultimas_semanas, 'probabilidade_venda'] = 0.05
    
    print(f'📊 Dados simulados criados: {dados_teste.shape}')

## 2. Análise Geral da Anomalia

In [ ]:
# Análise por semana
print('📊 Análise das previsões por semana:')
print('=' * 60)

analise_semanal = dados_teste.groupby('semana').agg({
    'quantidade_prevista': ['mean', 'std', 'min', 'max', lambda x: (x == 0).sum()],
    'probabilidade_venda': ['mean', 'std'],
    'pdv_id': 'count'
}).round(4)

analise_semanal.columns = ['qty_mean', 'qty_std', 'qty_min', 'qty_max', 'qty_zeros', 'prob_mean', 'prob_std', 'total_registros']
analise_semanal['pct_zeros'] = (analise_semanal['qty_zeros'] / analise_semanal['total_registros'] * 100).round(1)

print(analise_semanal)

# Identificar semanas problemáticas
semanas_problematicas = analise_semanal[analise_semanal['pct_zeros'] > 50].index
print(f'\n🚨 Semanas com >50% de previsões zero: {len(semanas_problematicas)}')
for semana in semanas_problematicas:
    pct = analise_semanal.loc[semana, 'pct_zeros']
    print(f'   📅 {semana.strftime("%Y-%m-%d")}: {pct}% zeros')

## 3. Diagnóstico Detalhado: Caso Específico

In [ ]:
# Selecionar um PDV e Produto para análise detalhada
# Escolher combinação que aparece em todas as semanas
combinacoes_completas = dados_teste.groupby(['pdv_id', 'produto_id']).size()
combinacoes_5_semanas = combinacoes_completas[combinacoes_completas == 5].index

if len(combinacoes_5_semanas) > 0:
    pdv_exemplo, produto_exemplo = combinacoes_5_semanas[0]
    print(f'🔍 Analisando PDV {pdv_exemplo} x Produto {produto_exemplo}')
    
    # Filtrar dados para este exemplo
    df_diagnostico = dados_teste[
        (dados_teste['pdv_id'] == pdv_exemplo) & 
        (dados_teste['produto_id'] == produto_exemplo)
    ].copy().sort_values('semana')
    
    print(f'📊 Registros encontrados: {len(df_diagnostico)}')
    
    # Mostrar evolução das principais variáveis
    colunas_diagnostico = ['semana', 'quantidade_prevista', 'probabilidade_venda']
    
    # Adicionar features de lag se existirem
    lag_features = [col for col in dados_teste.columns if 'lag' in col.lower()]
    media_features = [col for col in dados_teste.columns if 'media' in col.lower()]
    
    features_importantes = lag_features + media_features
    if features_importantes:
        colunas_diagnostico.extend(features_importantes[:10])  # Primeiras 10 features
    
    # Filtrar apenas colunas que existem
    colunas_existentes = [col for col in colunas_diagnostico if col in df_diagnostico.columns]
    
    print('\n📋 Evolução das Features Principais:')
    print('=' * 80)
    if len(colunas_existentes) > 2:
        print(df_diagnostico[colunas_existentes].to_string(index=False))
    else:
        print(df_diagnostico[['semana', 'quantidade_prevista', 'probabilidade_venda']].to_string(index=False))
        
else:
    print('⚠️ Nenhuma combinação PDV x Produto encontrada em todas as 5 semanas')
    print('💡 Isso pode indicar problemas na geração do grid de teste')

## 4. Análise Visual da Anomalia

In [ ]:
# Criar visualizações
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Diagnóstico da Anomalia de Previsão', fontsize=16, fontweight='bold')

# 1. Distribuição de previsões por semana
axes[0,0].boxplot([dados_teste[dados_teste['semana'] == semana]['quantidade_prevista'].values 
                   for semana in sorted(dados_teste['semana'].unique())],
                  labels=[s.strftime('%m-%d') for s in sorted(dados_teste['semana'].unique())])
axes[0,0].set_title('Distribuição de Quantidade Prevista por Semana')
axes[0,0].set_ylabel('Quantidade Prevista')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Percentual de zeros por semana
pct_zeros_semana = dados_teste.groupby('semana')['quantidade_prevista'].apply(lambda x: (x == 0).mean() * 100)
axes[0,1].bar(range(len(pct_zeros_semana)), pct_zeros_semana.values, color='red', alpha=0.7)
axes[0,1].set_title('Percentual de Previsões Zero por Semana')
axes[0,1].set_ylabel('% Previsões Zero')
axes[0,1].set_xticks(range(len(pct_zeros_semana)))
axes[0,1].set_xticklabels([s.strftime('%m-%d') for s in pct_zeros_semana.index], rotation=45)

# 3. Distribuição de probabilidades por semana
axes[1,0].boxplot([dados_teste[dados_teste['semana'] == semana]['probabilidade_venda'].values 
                   for semana in sorted(dados_teste['semana'].unique())],
                  labels=[s.strftime('%m-%d') for s in sorted(dados_teste['semana'].unique())])
axes[1,0].set_title('Distribuição de Probabilidade de Venda por Semana')
axes[1,0].set_ylabel('Probabilidade de Venda')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Evolução do caso específico (se disponível)
if len(combinacoes_5_semanas) > 0:
    axes[1,1].plot(df_diagnostico['semana'], df_diagnostico['quantidade_prevista'], 'o-', label='Quantidade Prevista', linewidth=2)
    ax2 = axes[1,1].twinx()
    ax2.plot(df_diagnostico['semana'], df_diagnostico['probabilidade_venda'], 's-', color='red', label='Prob. Venda', alpha=0.7)
    axes[1,1].set_title(f'Caso Específico: PDV {pdv_exemplo} x Produto {produto_exemplo}')
    axes[1,1].set_ylabel('Quantidade Prevista')
    ax2.set_ylabel('Probabilidade de Venda', color='red')
    axes[1,1].legend(loc='upper left')
    ax2.legend(loc='upper right')
    axes[1,1].tick_params(axis='x', rotation=45)
else:
    axes[1,1].text(0.5, 0.5, 'Caso específico não disponível', ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Caso Específico: Não Disponível')

plt.tight_layout()
plt.show()

## 5. Conclusões e Recomendações

In [ ]:
print('🎯 CONCLUSÕES DO DIAGNÓSTICO')
print('=' * 60)

# Calcular estatísticas da anomalia
primeiras_3_semanas = sorted(dados_teste['semana'].unique())[:3]
ultimas_2_semanas = sorted(dados_teste['semana'].unique())[-2:]

pct_zeros_inicio = dados_teste[dados_teste['semana'].isin(primeiras_3_semanas)]['quantidade_prevista'].apply(lambda x: x == 0).mean() * 100
pct_zeros_fim = dados_teste[dados_teste['semana'].isin(ultimas_2_semanas)]['quantidade_prevista'].apply(lambda x: x == 0).mean() * 100

print(f'📊 Primeiras 3 semanas: {pct_zeros_inicio:.1f}% de previsões zero')
print(f'📊 Últimas 2 semanas: {pct_zeros_fim:.1f}% de previsões zero')
print(f'📈 Aumento: {pct_zeros_fim - pct_zeros_inicio:.1f} pontos percentuais')

print('\n🔍 CAUSA PROVÁVEL DA ANOMALIA:')
print('\n1. **Dependência Excessiva de Features de Lag**:')
print('   • Modelo depende muito das features quantidade_lag_1 a quantidade_lag_4')
print('   • Após 4 semanas de previsão, estas features ficam sem histórico')
print('   • Pipeline não tem dados "reais" para alimentar os lags futuros')

print('\n2. **Degradação das Features de Média Móvel**:')
print('   • Features como quantidade_media_4w também perdem qualidade')
print('   • Calculadas com base em previsões, não dados reais')

print('\n3. **Ausência de Features de Longo Prazo**:')
print('   • Falta de lag de 52 semanas (sazonalidade anual)')
print('   • Sem features que capturem padrões anuais')

print('\n💡 RECOMENDAÇÕES PARA CORREÇÃO:')
print('\n🔧 **Implementação Imediata**:')
print('   1. Adicionar features EWMA (menos sensíveis a valores únicos)')
print('   2. Incluir features de calendário (semana do ano, mês)')
print('   3. Adicionar lag de 52 semanas se dados de 2021 disponíveis')
print('   4. Features de preço relativo por categoria')

print('\n🎯 **Estratégia de Validação**:')
print('   1. Testar threshold otimizado para WMAPE')
print('   2. Validar no conjunto de validação (últimas 5 semanas de 2022)')
print('   3. Monitorar distribuição de previsões por semana')

print('\n🚀 **Próximos Passos**:')
print('   1. Implementar features avançadas no notebook 10')
print('   2. Otimizar threshold no notebook 11b')
print('   3. Atualizar pipeline final no notebook 13')
print('   4. Executar este diagnóstico novamente para validar correções')

print('\n✅ Diagnóstico concluído - execute as correções nos notebooks seguintes!')